In [64]:
import tensorflow as tf
import numpy as np
from music21 import stream, instrument, note, chord
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint

try:
  import google.colab
  IS_ON_GOOGLE_COLAB = True
except:
  IS_ON_GOOGLE_COLAB = False

if IS_ON_GOOGLE_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

In [65]:
print(tf.test.is_gpu_available())
tf.config.list_physical_devices('GPU')

True


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [66]:
import glob
import os
import pickle

from music21 import converter, pitch, interval, instrument, note, chord
import tensorflow as tf
# Define save directory
from music21.key import Key
import numpy as np
from pathlib import Path

midi_dir = './midi_songs/'

def get_current_datetime():
    from datetime import datetime
    now = datetime.now()
    dt_name = now.strftime("%m_%d_%Y__%H_%M_%S")
    return dt_name

if IS_ON_GOOGLE_COLAB:
  FOLDER_ROOT = os.path.join("content", "drive", "MyDrive", "magisterka", "SheetMusicGenerator2")
else:
  FOLDER_ROOT = os.path.join(".")

TEST_RUN = False
NORMALIZE_NOTES = True
NORMALIZATION_BOUNDARIES = [3, 4]

EPOCHS = 50

AUTOENCODER = "AUTOENCODER"
MODEL_NAME = AUTOENCODER

MODEL_FOLDER_ROOT=os.path.join(FOLDER_ROOT, MODEL_NAME)
CURR_DT = get_current_datetime()
MODEL_DIR_PATH = os.path.join(MODEL_FOLDER_ROOT, "generated_models")
OCCURENCES = os.path.join(MODEL_FOLDER_ROOT, "data", "occurences")
DATA_NOTES_DIR = os.path.join(MODEL_FOLDER_ROOT, "data", "notes")
DATA_DURATIONS_DIR = os.path.join(MODEL_FOLDER_ROOT, "data", "durations")
MIDI_SONGS_DIR = os.path.join(FOLDER_ROOT, "midi_songs")
# MIDI_SONGS_DIR = os.path.join(FOLDER_ROOT, "midi_songs_smaller")
MIDI_GENERATED_DIR = os.path.join(MODEL_FOLDER_ROOT, "midi_generated")
MIDI_SONGS_REGEX = os.path.join(MIDI_SONGS_DIR, "*.mid")
CHECKPOINTS_DIR = os.path.join(MODEL_FOLDER_ROOT, "checkpoints"  )
CHECKPOINT = os.path.join(CHECKPOINTS_DIR, str(CURR_DT))
LOGS_DIR = os.path.join(MODEL_FOLDER_ROOT, "logs")
LOG = os.path.join(LOGS_DIR, str(CURR_DT))

all_paths = [MODEL_DIR_PATH, MODEL_NAME, OCCURENCES, DATA_NOTES_DIR, DATA_DURATIONS_DIR, MIDI_GENERATED_DIR, CHECKPOINTS_DIR, CHECKPOINT, LOGS_DIR, LOG]
for path in all_paths:
    Path(path).mkdir(parents=True, exist_ok=True)

def create_train_data():
# Create empty list for scores
    originalScores = []

    # Load and make list of stream objects
    for song in glob.glob(MIDI_SONGS_REGEX):
        print("Parsing song: " + str(song))
        score = converter.parse(song)
        originalScores.append(score)

    # Define empty lists of lists
    originalChords = [[] for _ in originalScores]
    originalDurations = [[] for _ in originalScores]
    originalKeys = []

    def transpose_amount(score):
        return -int(score.chordify().analyze('key').tonic.ps % 12)

    def monophonic(stream):
        try:
            length = len(instrument.partitionByInstrument(stream).parts)
        except:
            length = 0
        return length == 1
    # Extract notes, chords, durations, and keys


    originalScores = [song.chordify() for song in originalScores]

    for i, song in enumerate(originalScores):

        # song.transpose
        transp_int = transpose_amount(song)
        originalKeys.append(str(song.analyze('key').transpose(transp_int)))
        for element in song:
            if isinstance(element, note.Note):
                originalChords[i].append(element.pitch.transpose(transp_int))
                originalDurations[i].append(element.duration.quarterLength)
            elif isinstance(element, chord.Chord):
                originalChords[i].append('.'.join(str(n.transpose(transp_int)) for n in element.pitches))
                originalDurations[i].append(element.duration.quarterLength)
        print(str(originalKeys[i]))

    cChords = [c for (c, k) in zip(originalChords, originalKeys) if (k == 'C major')]
    cDurations = [c for (c, k) in zip(originalDurations, originalKeys) if (k == 'C major')]
    # Map unique chords to integers
    uniqueChords = np.unique([i for s in originalChords for i in s])
    chordToInt = dict(zip(uniqueChords, list(range(0, len(uniqueChords)))))

    # Map unique durations to integers
    uniqueDurations = np.unique([i for s in originalDurations for i in s])
    durationToInt = dict(zip(uniqueDurations, list(range(0, len(uniqueDurations)))))

    # Print number of unique notes and chords
    print(len(uniqueChords))

    # Print number of unique durations
    print(len(uniqueDurations))

    intToChord = {i: c for c, i in chordToInt.items()}
    intToDuration = {i: c for c, i in durationToInt.items()}

    # Define sequence length
    sequenceLength = 32

    # Define empty arrays for train data
    trainChords = []
    trainDurations = []

    # Construct training sequences for chords and durations
    for s in range(len(cChords)):
        chordList = [chordToInt[c] for c in cChords[s]]
        durationList = [durationToInt[d] for d in cDurations[s]]
        for i in range(len(chordList) - sequenceLength):
            trainChords.append(chordList[i:i+sequenceLength])
            trainDurations.append(durationList[i:i+sequenceLength])

    with open(DATA_NOTES_DIR + "/notes", 'wb') as filepath:
        pickle.dump(cChords, filepath)

    with open(DATA_DURATIONS_DIR + "/durations", 'wb') as filepath:
        pickle.dump(cDurations, filepath)
#
#     trainChords = tf.keras.utils.to_categorical(trainChords).transpose(0,2,1)
#
# # Convert data to numpy array of type float
#     trainChords = np.array(trainChords, np.float)

# Flatten sequence of chords into single dimension
        # Convert to one-hot encoding and swap chord and sequence dimensions
    print("trainChordsFlat: " + str(trainChords))
    trainChords = tf.keras.utils.to_categorical(trainChords).transpose(0, 2, 1)

    # Convert data to numpy array of type float
    trainChords = np.array(trainChords, np.float32)

    nSamples = trainChords.shape[0]
    nChords = trainChords.shape[1]
    inputDim = nChords * sequenceLength
    # Flatten sequence of chords into single dimension
    trainChordsFlat = trainChords.reshape(nSamples, inputDim)

    return trainChordsFlat, inputDim, trainDurations, sequenceLength, intToChord, intToDuration, nChords

# if __name__ == "__main__":
#     create_train_data()
# # Convert to one-hot encoding and swap chord and sequence dimensions

In [67]:
class MusicAutoencoder():

    def __init__(self, latentDim, trainChordsFlat, inputDim, sequenceLength, intToChord, intToDuration, nChords):
        self.trainChordsFlat = trainChordsFlat
        self.inputDim = inputDim
        self.sequenceLength = sequenceLength
        self.nChords = nChords
        self.latentDim = latentDim
        self.intToChord = intToChord
        self.intToDuration = intToDuration
        self.encoder = None
        self.decoder = None
        self.model = self.autoencoder(inputDim, latentDim)


    def autoencoder(self, inputDim, latentDim):
        # Define encoder input shape
        encoderInput = tf.keras.layers.Input(shape = (inputDim))

        # Define decoder input shape
        latent = tf.keras.layers.Input(shape =(latentDim))

        # Define dense encoding layer connecting input to latent vector
        encoded = tf.keras.layers.Dense(latentDim, activation = 'tanh')(encoderInput)

        # Define dense decoding layer connecting latent vector to output
        decoded = tf.keras.layers.Dense(inputDim, activation = 'sigmoid')(latent)

        # Define the encoder and decoder models
        self.encoder = tf.keras.Model(encoderInput, encoded)
        self.decoder = tf.keras.Model(latent, decoded)

        # Define autoencoder model
        autoencoder = tf.keras.Model(encoderInput, self.decoder(encoded))
        return autoencoder



    def train(self, checkpoint_path=None):
        # Define number of samples, chords and notes, and input dimension
        # filepath = CHECKPOINTS + "weights-improvement-{epoch:02d}-{loss:.4f}-{categorical_accuracy:.4f}-bigger.hdf5"
        # filepath = "weights-improvement-epoch:{epoch:02d}-loss:{loss:.4f}-cat_acc:{categorical_accuracy:.4f}.hdf5"
        
        print(str("Current datatime: " + CURR_DT))

        if checkpoint_path:
            self.model.load_weights(checkpoint_path)

        filepath = os.path.join(CHECKPOINT, "epoch={epoch:02d}-loss={loss:.4f}-acc={binary_accuracy:.4f}.hdf5")

        # filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
        checkpoint = ModelCheckpoint(
            filepath,
            monitor='binary_accuracy',
            verbose=0,
            save_best_only=True,
            mode='max'
        )
        log = tf.keras.callbacks.TensorBoard(log_dir=LOG),

        callbacks_list = [checkpoint, log]
        # history = self.model.fit(network_input, network_output, epochs=EPOCHS, batch_size=128, callbacks=callbacks_list)
        # model.save(MODEL_DIR_PATH + MODEL_NAME + "_" + CURR_DT + ".hdf5")

        self.model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.01), metrics=[tf.keras.metrics.BinaryAccuracy()])
        # self.model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.01), metrics=["accuracy"])
        # Train autoencoder
        self.model.summary()
        print(MODEL_DIR_PATH + MODEL_NAME + "_" + CURR_DT + ".hdf5")
        # history = self.model.fit(self.trainChordsFlat, self.trainChordsFlat, epochs=1)
        # history = self.model.fit(self.trainChordsFlat, self.trainChordsFlat, epochs=500, callbacks=callbacks_list, batch_size=8)
        tensor_dataset = tf.data.Dataset.from_tensors((self.trainChordsFlat, self.trainChordsFlat))
        for el in tensor_dataset:
            print(el)

        history = self.model.fit(tensor_dataset, epochs=EPOCHS, callbacks=callbacks_list)
        print(history.history)
        print(MODEL_DIR_PATH + MODEL_NAME + "_" + CURR_DT + ".hdf5")
        self.model.save(os.path.join(MODEL_DIR_PATH, MODEL_NAME + "_" + CURR_DT + ".hdf5"))


    def generateChords(self):
        generatedChords = self.decoder(np.random.normal(size=(1, self.latentDim))).numpy().reshape(self.nChords, self.sequenceLength).argmax(0)
        generatedStream = stream.Stream()

        generatedStream.append(instrument.Piano())
        chordSequence = [self.intToChord[c] for c in generatedChords]
        # Append notes and chords to stream object
        for j in range(len(chordSequence)):
            try:
                generatedStream.append(note.Note(chordSequence[j].replace('.', ' ')))
            except:
                generatedStream.append(chord.Chord(chordSequence[j].replace('.', ' ')))

        generatedStream.write('midi', fp=MIDI_GENERATED_DIR + 'autoencoder.mid')
        # return generatedChords

In [68]:
class ModelFactory:
    def factory(self, model_type):
        if model_type == AUTOENCODER:
          trainChordsFlat, inputDim, trainDurations, sequenceLength, intToChord, intToDuration, nChords = create_train_data()
          return MusicAutoencoder(2, trainChordsFlat, inputDim, sequenceLength, intToChord, intToDuration, nChords)

In [ ]:
modelFactory = ModelFactory()
musicAutoencoder = modelFactory.factory(MODEL_NAME)
musicAutoencoder.train()
musicAutoencoder.generateChords()

Parsing song: .\midi_songs\0fithos.mid
Parsing song: .\midi_songs\8.mid
Parsing song: .\midi_songs\ahead_on_our_way_piano.mid
Parsing song: .\midi_songs\AT.mid
Parsing song: .\midi_songs\balamb.mid
Parsing song: .\midi_songs\bcm.mid
Parsing song: .\midi_songs\BlueStone_LastDungeon.mid
Parsing song: .\midi_songs\braska.mid
Parsing song: .\midi_songs\caitsith.mid
Parsing song: .\midi_songs\Cids.mid
Parsing song: .\midi_songs\cosmo.mid
Parsing song: .\midi_songs\costadsol.mid
Parsing song: .\midi_songs\dayafter.mid
Parsing song: .\midi_songs\decisive.mid
Parsing song: .\midi_songs\dontbeafraid.mid
Parsing song: .\midi_songs\DOS.mid
Parsing song: .\midi_songs\electric_de_chocobo.mid
Parsing song: .\midi_songs\Eternal_Harvest.mid
Parsing song: .\midi_songs\EyesOnMePiano.mid
Parsing song: .\midi_songs\ff11_awakening_piano.mid
Parsing song: .\midi_songs\ff1battp.mid
Parsing song: .\midi_songs\FF3_Battle_(Piano).mid
Parsing song: .\midi_songs\FF3_Third_Phase_Final_(Piano).mid
Parsing song: .\m